<a href="https://colab.research.google.com/github/Adnan4555/ADNAN-4555/blob/GEN-AI/Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
classifier("we are very happy to show you the transformers library")

[{'label': 'POSITIVE', 'score': 0.9998044371604919}]

In [ ]:
classifier("the pizza is not that great but crust is good")

[{'label': 'POSITIVE', 'score': 0.9996163845062256}]

In [ ]:
results = classifier(["we are very happy to show you the transformers library","we hope you dont hate it"])
for result in results:
  print(f"label:{result['label']},with score:{round(result['score'],4)}")

label:POSITIVE,with score:0.9998
label:NEGATIVE,with score:0.96


In [ ]:
classifier = pipeline("sentiment-analysis",model='nlptown/bert-base-multilingual-uncased-sentiment')

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
classifier("esperamos que no lo odie")

[{'label': '3 stars', 'score': 0.2881225049495697}]

#----------------------------------------------------------------------------------------------------

In [ ]:
from transformers import BertTokenizer,BertForMaskedLM
import torch

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

In [ ]:
#sample etxt with missing word
text = "BERT is designed to [MASK] the context of a word in a sentence"


In [ ]:
input_ids = tokenizer(text,return_tensors='pt')

In [ ]:
input_ids

{'input_ids': tensor([[  101, 14324,  2003,  2881,  2000,   103,  1996,  6123,  1997,  1037,
          2773,  1999,  1037,  6251,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
#find the position of [MASK] token
mask_position = torch.where(input_ids['input_ids'][0]==tokenizer.mask_token_id)
mask_position

(tensor([5]),)

In [ ]:
#foreward pass.get prediction for the masked token
with torch.no_grad():
  output=model(**input_ids)

In [ ]:
output.logits

tensor([[[ -6.6673,  -6.6071,  -6.6109,  ...,  -5.9273,  -5.7575,  -4.1085],
         [ -4.8093,  -4.7351,  -4.8461,  ...,  -4.5364,  -5.1227,  -5.7692],
         [ -9.3709,  -9.1317,  -9.4016,  ...,  -7.9609,  -6.6966,  -7.7601],
         ...,
         [-12.6410, -12.7944, -12.5528,  ..., -10.7899,  -8.8492, -10.3958],
         [ -9.8646,  -9.9855, -10.0537,  ...,  -8.8559,  -9.6661,  -7.8782],
         [ -9.7484,  -9.5575,  -9.8182,  ...,  -7.3714,  -8.0097,  -7.6730]]])

In [ ]:
masked_token_logits = output.logits[0,mask_position,:]

In [ ]:
masked_token_logits

tensor([[-2.9975, -2.8993, -3.0351,  ..., -4.0165, -3.5004, -4.3098]])

In [ ]:
#find the top 5 predicted tokens and their corresponding words
top_tokens = torch.topk(masked_token_logits,k=5,dim=1).indices[0].tolist()
top_words = [tokenizer.decode([token]) for token in top_tokens]


In [ ]:
print("Input Text:",text)
print("Predicted Words:",top_words)

Input Text: BERT is designed to [MASK] the context of a word in a sentence
Predicted Words: ['understand', 'change', 'fit', 'capture', 'convey']
